
## Task 3 – Requirement 1: Các ràng buộc của Sudoku CSP

### 1. Biểu diễn biến mệnh đề

Ta biểu diễn Sudoku 9×9 dưới dạng bài toán thỏa ràng buộc (CSP) bằng các biến mệnh đề như sau:

- `X[r,c,v]`: biến mệnh đề mang ý nghĩa  
  “ô ở **hàng** r, **cột** c nhận **giá trị** v”.
- Chỉ số:
  - `r = 1..9` (hàng),
  - `c = 1..9` (cột),
  - `v = 1..9` (giá trị).

Như vậy, mỗi ô `(r,c)` có **9 biến mệnh đề**:

```text
X[r,c,1], X[r,c,2], ..., X[r,c,9]
```

Trong code, mỗi biến được ánh xạ thành một số nguyên duy nhất bằng hàm:

```text
var_id(r, c, v) = 81 * (r - 1) + 9 * (c - 1) + v    (chạy từ 1 đến 729)
```

---

### 2. Nhóm ràng buộc trong Sudoku CSP

Dưới đây là các loại ràng buộc (constraints) mình sử dụng, kèm theo ví dụ minh họa.
Mỗi ràng buộc đều sẽ được chuyển thành các mệnh đề CNF (danh sách các literal nguyên) trong chương trình Python.

---

#### 2.1. Ràng buộc “mỗi ô đúng 1 giá trị”

**Ý nghĩa:**  
Mỗi ô Sudoku phải chứa **chính xác một** số từ 1 đến 9.

Ta tách thành 2 phần:

1. **Ít nhất một giá trị** (at least one):

> Với mọi `r, c`:  
> một trong các biến `X[r,c,1]`, ..., `X[r,c,9]` phải đúng.

Biểu diễn logic:

```text
X[r,c,1] OR X[r,c,2] OR ... OR X[r,c,9]
```

2. **Không nhiều hơn một giá trị** (at most one):

> Với mọi `r, c` và mọi cặp giá trị khác nhau `v != w`:  
> không thể vừa `X[r,c,v]` đúng **và** `X[r,c,w]` đúng.

Biểu diễn logic:

```text
(NOT X[r,c,v]) OR (NOT X[r,c,w])
```

**Ví dụ minh họa (ô hàng 1, cột 1):**

- Ràng buộc “ít nhất một giá trị”:

```text
X[1,1,1] OR X[1,1,2] OR X[1,1,3] OR X[1,1,4] OR
X[1,1,5] OR X[1,1,6] OR X[1,1,7] OR X[1,1,8] OR X[1,1,9]
```

- Một vài ràng buộc “không nhiều hơn một giá trị”:

```text
(NOT X[1,1,1]) OR (NOT X[1,1,2])
(NOT X[1,1,1]) OR (NOT X[1,1,3])
...
(NOT X[1,1,8]) OR (NOT X[1,1,9])
```

**Liên hệ code:**  
Trong lớp `CNFEncoder`, hàm `encode_cells()` gọi `exactly_one` cho danh sách:

```python
[var_id(r, c, v) for v in range(1, 10)]
```

để sinh đúng các ràng buộc trên cho từng ô.

---

#### 2.2. Ràng buộc hàng (row constraints)

**Ý nghĩa:**  
Trên **mỗi hàng** `r`, với **mỗi giá trị** `v`, số `v` chỉ được xuất hiện **đúng một lần**.

Cũng tách thành 2 phần:

1. **Ít nhất một lần** trên hàng `r`:

> Với mọi `r, v`:  
> số `v` phải xuất hiện ở ít nhất một cột trong hàng `r`.

Biểu diễn logic:

```text
X[r,1,v] OR X[r,2,v] OR ... OR X[r,9,v]
```

2. **Không nhiều hơn một lần**:

> Với mọi `r, v` và mọi cặp cột `c1 != c2`:  
> không thể vừa ô `(r,c1)` và `(r,c2)` đều mang giá trị `v`.

Biểu diễn logic:

```text
(NOT X[r,c1,v]) OR (NOT X[r,c2,v])
```

**Ví dụ minh họa (hàng 1, số 5):**

- Số 5 phải xuất hiện ở **ít nhất một** ô trong hàng 1:

```text
X[1,1,5] OR X[1,2,5] OR X[1,3,5] OR X[1,4,5] OR
X[1,5,5] OR X[1,6,5] OR X[1,7,5] OR X[1,8,5] OR X[1,9,5]
```

- Không thể vừa ô `(1,1)` và ô `(1,3)` đều là số 5:

```text
(NOT X[1,1,5]) OR (NOT X[1,3,5])
```

**Liên hệ code:**  
Trong `CNFEncoder.encode_rows()`, ta gọi:

```python
self.exactly_one([var_id(r, c, v) for c in range(1, 10)])
```

cho từng hàng `r` và từng giá trị `v`, tạo ra đúng các ràng buộc hàng như trên.

---

#### 2.3. Ràng buộc cột (column constraints)

**Ý nghĩa:**  
Trên **mỗi cột** `c`, với **mỗi giá trị** `v`, số `v` chỉ được xuất hiện **đúng một lần**.

1. **Ít nhất một lần** trên cột `c`:

> Với mọi `c, v`:  
> số `v` phải xuất hiện ở ít nhất một hàng trong cột `c`.

Biểu diễn logic:

```text
X[1,c,v] OR X[2,c,v] OR ... OR X[9,c,v]
```

2. **Không nhiều hơn một lần**:

> Với mọi `c, v` và mọi cặp hàng `r1 != r2`:  
> không thể vừa ô `(r1,c)` và `(r2,c)` đều mang giá trị `v`.

Biểu diễn logic:

```text
(NOT X[r1,c,v]) OR (NOT X[r2,c,v])
```

**Ví dụ minh họa (cột 3, số 7):**

- Số 7 phải xuất hiện ở **ít nhất một** ô trong cột 3:

```text
X[1,3,7] OR X[2,3,7] OR X[3,3,7] OR X[4,3,7] OR
X[5,3,7] OR X[6,3,7] OR X[7,3,7] OR X[8,3,7] OR X[9,3,7]
```

- Không thể ô `(2,3)` và ô `(9,3)` cùng là 7:

```text
(NOT X[2,3,7]) OR (NOT X[9,3,7])
```

**Liên hệ code:**  
Trong `CNFEncoder.encode_cols()`, ta gọi:

```python
self.exactly_one([var_id(r, c, v) for r in range(1, 10)])
```

cho từng cột `c` và từng giá trị `v`.

---

#### 2.4. Ràng buộc block 3×3 (subgrid constraints)

Bảng Sudoku 9×9 được chia thành 9 block 3×3.  
Trong mỗi block 3×3, **mỗi giá trị** `v` chỉ được xuất hiện **đúng một lần**.

Gọi `br, bc = 0,1,2` là chỉ số block theo dòng và cột.

- Các hàng trong block `(br, bc)`:

```text
r = 3*br + 1, 3*br + 2, 3*br + 3
```

- Các cột trong block `(br, bc)`:

```text
c = 3*bc + 1, 3*bc + 2, 3*bc + 3
```

1. **Ít nhất một lần** trong block:

> Với mọi `br, bc, v`:  
> số `v` phải xuất hiện ở ít nhất một ô trong block đó.

Biểu diễn logic (dạng khái quát):

```text
OR_{r,c trong block(br,bc)}  X[r,c,v]
```

2. **Không nhiều hơn một lần** trong block:

> Với mọi `br, bc, v` và mọi cặp ô khác nhau `(r1,c1) != (r2,c2)` cùng thuộc block:  
> không thể vừa ô `(r1,c1)` và `(r2,c2)` đều mang giá trị `v`.

Biểu diễn logic:

```text
(NOT X[r1,c1,v]) OR (NOT X[r2,c2,v])
```

**Ví dụ minh họa:** block trên cùng bên trái (br = 0, bc = 0), số 9:

- Các ô trong block này là (1..3, 1..3).  
  Ràng buộc “ít nhất một ô là 9”:

```text
X[1,1,9] OR X[1,2,9] OR X[1,3,9] OR
X[2,1,9] OR X[2,2,9] OR X[2,3,9] OR
X[3,1,9] OR X[3,2,9] OR X[3,3,9]
```

**Liên hệ code:**  
Trong `CNFEncoder.encode_blocks()`, ta tính các chỉ số hàng/cột của từng block:

```python
rows = range(1 + 3*br, 4 + 3*br)
cols = range(1 + 3*bc, 4 + 3*bc)
```

và gọi:

```python
self.exactly_one([var_id(r, c, v) for r in rows for c in cols])
```

cho mỗi block `(br,bc)` và mỗi giá trị `v`.

---

#### 2.5. Ràng buộc các ô cho trước (clues)

**Ý nghĩa:**  
Những ô đã có số trong đề Sudoku phải được giữ nguyên trong nghiệm.

Nếu đề ban đầu có số `d` tại ô `(r,c)`, ta thêm ràng buộc:

```text
X[r,c,d]
```

tức là biến `X[r,c,d]` **bắt buộc TRUE** (ô `(r,c)` luôn là `d` trong nghiệm).

**Ví dụ minh họa:**  
Giả sử trong đề Sudoku, ô `(1,4)` được cho sẵn là giá trị `2`.  
Ta có ràng buộc clues:

```text
X[1,4,2]
```

Điều này đảm bảo mọi nghiệm hợp lệ của bài toán CSP đều giữ nguyên ô `(1,4) = 2`.

**Liên hệ code:**  
Trong `CNFEncoder.encode_clues()`, ta duyệt qua toàn bộ ma trận đầu vào `grid`:

```python
for r in range(1, 10):
    for c in range(1, 10):
        v = self.grid[r-1][c-1]
        if 1 <= v <= 9:
            self.add([var_id(r, c, v)])
```

Với mỗi ô có giá trị `v` khác 0, ta sinh một clause đơn `[var_id(r,c,v)]`
tương ứng với ràng buộc `X[r,c,v]` ở trên.

---

### 3. Tóm tắt Requirement 1

- Đã xác định biến mệnh đề `X[r,c,v]` cho toàn bộ bảng Sudoku 9×9.
- Đã liệt kê đủ các nhóm ràng buộc của CSP:
  - mỗi ô đúng 1 giá trị,
  - mỗi hàng, mỗi số xuất hiện đúng 1 lần,
  - mỗi cột, mỗi số xuất hiện đúng 1 lần,
  - mỗi block 3×3, mỗi số xuất hiện đúng 1 lần,
  - giữ nguyên các ô cho trước.
- Với mỗi nhóm ràng buộc, đã đưa ra ví dụ minh họa cụ thể
  (chọn ô, hàng, cột, block và giá trị cụ thể để viết thành mệnh đề).
- Các ràng buộc này chính là cơ sở để sinh ra các clause CNF
  trong lớp `CNFEncoder` và sau đó đưa vào SAT solver (Glucose3) để tìm nghiệm Sudoku.
